In [0]:
from collections import Counter
import re
import operator
import random
import itertools

class CYK:
    def __init__(self, all_rule_keys):
        self.all_rule_keys = all_rule_keys
        self.all_rules = all_rules
        self.words_and_tags = words_and_tags
        self.rules_and_tags = rules_and_tags

    def rules(path):
        with open(path) as f:
          # Read lines and store them in all_lines array 
            all_lines = f.readlines()
            rule_set = []
            all_rule_keys = []
            for line in all_lines:
                # If line is not empty line or comment line split into words
                # Else ignore it
                if line != '\n' and line[0] != '#':
                    words = tuple(line.split())
                    if 'true' in words:
                      new_word = [words[0],words[1]+' '+words[2]+' '+words[3]+' '+words[4],words[5],words[6]]
                      words =tuple(new_word) 
                    else:
                        for word in words:
                            # delete comment lines
                            if '#' in word:
                                words = tuple(words[0:words.index(word)])
                                break
                    if words[0] not in all_rule_keys:
                          all_rule_keys.append(words[0])
                    rule_set.append(words)
            all_rules = {}
            for i in range(0,len(rule_set)):
                if len(rule_set[i]) == 4:
                    value = []
                    value.append(rule_set[i][1] + '|' + rule_set[i][2] + '|' + rule_set[i][3])
                    if rule_set[i][0] in all_rules.keys():
                        all_rules[rule_set[i][0]].append(value[0]) 
                    else:
                        all_rules[rule_set[i][0]] = value
                elif len(rule_set[i]) == 3:
                    value = []
                    value.append(rule_set[i][1] + '|' + rule_set[i][2]) 
                    if rule_set[i][0] in all_rules.keys():
                        all_rules[rule_set[i][0]].append(value[0]) 
                    else:
                        all_rules[rule_set[i][0]] = value
                elif len(rule_set[i]) == 2:
                    value = []
                    value.append(rule_set[i][1]) 
                    if rule_set[i][0] in all_rules.keys():
                        all_rules[rule_set[i][0]].append(value[0]) 
                    else:
                        all_rules[rule_set[i][0]] = value
            CYK.all_rule_keys = all_rule_keys 
            CYK.all_rules = all_rules
            return all_rules
            
    def recursive_function(rule_set, sentence, next_tag):
        if next_tag not in rule_set.keys():
          sentence.append(next_tag)
          return sentence
        liste = rule_set[next_tag]
        choosen = random.choice(liste)
        splitted_choosen = choosen.split('|')
        if choosen.count('|') == 0:
          if choosen in rule_set.keys():
              CYK.recursive_function(rule_set, sentence, choosen)
          else:
              sentence.append(choosen)
          return sentence
        elif choosen.count('|') is 1:
          CYK.recursive_function(rule_set, sentence, splitted_choosen[0])
          CYK.recursive_function(rule_set, sentence, splitted_choosen[1])
          return sentence
        elif choosen.count('|') is 2:
          CYK.recursive_function(rule_set, sentence, splitted_choosen[0])
          CYK.recursive_function(rule_set, sentence, splitted_choosen[1])
          CYK.recursive_function(rule_set, sentence, splitted_choosen[2])
          return sentence

    def randsentence(rule_set, file):
        generated_sentences = []
        generated_sentences_list = []
        for n in range(0,15):
            sentence = []
            sentence = CYK.recursive_function(rule_set, sentence,  'ROOT')
            sentence_string = ''
            for i in range(0, len(sentence)):
              sentence_string = sentence_string + " " + sentence[i]
            generated_sentences.append(sentence_string)
            generated_sentences_list.append(sentence)
         
        for i in range(0,len(generated_sentences)):
            print("Sentence ",i+1," : " ,generated_sentences[i])
            file.write(generated_sentences[i])
            file.write("\n")
        
        return generated_sentences_list

    def CYKParser(sentence):
        all_matrix = []
        sentence_length = len(sentence)
        matrix = [['' for x in range(0,sentence_length)] for y in range(0,sentence_length)]
        for i in range(0,sentence_length): #ROWS
          for j in range(0, sentence_length-i-1):# COLUMNS
              if i == 0:
                if CYK.words_and_tags[sentence[j]] == 'Pronoun':
                    matrix[i][j] = 'NP'
                else:
                    matrix[i][j] = CYK.words_and_tags[sentence[j]]
              else:                       
                key = matrix[i-1][j]+'|'+matrix[i-1][j+1]
                
                try:
                  matrix[i][j] = CYK.rules_and_tags[key]
                except:
                  matrix[i][j] = ""        
          
          

# Input File
rule_set = CYK.rules("/content/drive/My Drive/cfg.gr")
# Output File
output_file= open("/content/drive/My Drive/output.txt","w+")
generated_sentences = CYK.randsentence(rule_set, output_file)
words_and_tags = {}
rules_and_tags = {}
for i in CYK.all_rules:
  for j in range(0, len(CYK.all_rules[i])):
      if CYK.all_rules[i][j].count('|') == 0:
          words_and_tags[CYK.all_rules[i][j]] = i
      else:
          rules_and_tags[CYK.all_rules[i][j]] = i

words_and_tags['.'] = '.'
words_and_tags['?'] = '?'
words_and_tags['!'] = '!'
words_and_tags['is it true that'] = 'is it true that'
CYK.words_and_tags = words_and_tags
CYK.rules_and_tags = rules_and_tags

for i in range(0,len(generated_sentences)):
    CYK.CYKParser(generated_sentences[i])
output_file.close()

Sentence  1  :   i from i under the mouse from you to me like you to that sandwich in me !
Sentence  2  :   every beautiful sandwich like it .
Sentence  3  :   the floor pickled every sandwich on me .
Sentence  4  :   that sandwich prefer me .
Sentence  5  :   a floor ate that mouse !
Sentence  6  :   is it true that it is i from you from that president to i on this sandwich on a old president to me to every mouse ?
Sentence  7  :   the sandwich pickled it on a sandwich .
Sentence  8  :   i from the floor on the floor in every sandwich wanted a pickle under me to that president on that pickle under it under me with i with you in you in i !
Sentence  9  :   it want it .
Sentence  10  :   a pickle pickled this old sandwich !
Sentence  11  :   is it true that it want i to this floor ?
Sentence  12  :   is it true that the pickle washed every floor on it to me from it to me ?
Sentence  13  :   a floor ate you !
Sentence  14  :   is it true that this sandwich to you ate me ?
Sentence  15  :

In [0]:
from google.colab import drive

drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/
